In [1]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from keybert import KeyBERT

In [2]:
nlp = spacy.load("en_core_web_sm")

In [3]:
sds_text = pd.read_csv('../data/sds_text.csv')

<IPython.core.display.Javascript object>

In [4]:
sds_text = sds_text.set_index('Unnamed: 0')

In [5]:
sds_text

,episode_name,length_episode,context_episode,guest_name,guest_info,text_episode,episode_number,episode_date,episode_day,host_episode,speaker,episode_split_text
Unnamed: 0,,,,,,,,,,,,
0,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,"Sep 10, 2016",Saturday,Kirril Eremenko,Kirill,This is episode number one with ex-chemical e...
0,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,"Sep 10, 2016",Saturday,Kirril Eremenko,Kirill,"Hey guys, welcome to the Podcast. I’ve got Ru..."
0,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,"Sep 10, 2016",Saturday,Kirril Eremenko,Ruben,Thank you! Thanks for having me over. I’m doi...
0,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,"Sep 10, 2016",Saturday,Kirril Eremenko,Kirill,Awesome. It’s great to hear you and for those...
0,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,"Sep 10, 2016",Saturday,Kirril Eremenko,Ruben,"Yeah sure. So, I’m the senior manager of anal..."
...,...,...,...,...,...,...,...,...,...,...,...,...
681,"Contextual A.I. for Adapting to Adversaries, ...",81,Data Science Artificial Intelligence,Matar Haller,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:00:05\tThis is episode number 6...,683,"May 30, 2023",Tuesday,Jon Krohn,Jon Krohn,"Yeah, right. Yeah, as I mean, it actually, it ..."
681,"Contextual A.I. for Adapting to Adversaries, ...",81,Data Science Artificial Intelligence,Matar Haller,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:00:05\tThis is episode number 6...,683,"May 30, 2023",Tuesday,Jon Krohn,Matar Haller,Happy to.
681,"Contextual A.I. for Adapting to Adversaries, ...",81,Data Science Artificial Intelligence,Matar Haller,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:00:05\tThis is episode number 6...,683,"May 30, 2023",Tuesday,Jon Krohn,Jon Krohn,"Nice. Well, yeah, so you mentioned potentially..."


In [6]:
sds_text.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46772 entries, 0 to 681
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   episode_name        46772 non-null  object
 1   length_episode      46772 non-null  int64 
 2   context_episode     46772 non-null  object
 3   guest_name          46772 non-null  object
 4   guest_info          46772 non-null  object
 5   text_episode        46772 non-null  object
 6   episode_number      46772 non-null  int64 
 7   episode_date        46772 non-null  object
 8   episode_day         46772 non-null  object
 9   host_episode        46772 non-null  object
 10  speaker             46496 non-null  object
 11  episode_split_text  46772 non-null  object
dtypes: int64(2), object(10)
memory usage: 4.6+ MB


In [8]:
sds_processed = pd.DataFrame(sds_text.groupby('episode_number')['episode_split_text'].agg(lambda x: ' '.join(x)))

<IPython.core.display.Javascript object>

In [12]:
sds_processed

,episode_split_text
episode_number,
1,This is episode number one with ex-chemical e...
2,This is session number two with machine learn...
3,This is episode number three with Nanophysics ...
4,This is episode four with business strategy e...
5,This is episode number five with forensics inv...
...,...
679,This is episode number 679 with George Matthew...
680,This is episode number 680 with Allegra Alessi...
681,"This is episode number 681 with Matt Harrison,..."


In [35]:
sds_processed['episode_split_text'].loc[500]

'Welcome to the SuperDataScience show. For this special 500th episode, we\'re going to enjoy a mind-expanding Yoga Nidra session led by the wonderful Yogi, Jes Allen. Jes Allen, can\'t believe you\'re on the SuperDataScience podcast. It\'s wonderful to have you here. I was thinking for a while, I was thinking for months when I noticed that episode 500 was coming up, and what was months ago a distant future. And now it\'s not that far away, episode 500 will be released very soon and we\'re recording it right now. And so I was thinking to myself for months, I was like, "What can I do that would be extra special for the audience for episode 500?" And time and again, the idea that kept coming back to me was you offering a yoga Nidra practice. So now here you are. Yeah. Thank you. I\'m so excited to be here. I\'m so happy to be here. So I\'ve known you for a long time. I started doing yoga practice, the poses of yoga in 2012, I was living in Singapore and once a week on Saturdays at this mi

In [36]:
#doc = 'This is episode number 682 with Mico Yuk, host of the Analytics On Fire podcast.Welcome back to the SuperDataScience Podcast. Today I\'m lucky to be joined by Mico Yuk. A straight shooter who pulls absolutely no punches in her assessment of, well of anything, but particularly about vendors in the data analytics space. Mico is host of the popular Analytics on Fire podcast, which has a cult following. She co-founded the BI Brainz group and analytics, consulting and solutions company that has taught over 15,000 students analytics, visualization, and data storytelling courses, including at major multinationals like Nestle, FedEx, and Proctor & Gamble. She authored the Data Visualization for Dummies book and is a sought-after keynote speaker and TV news commentator.In this episode, Mico details her BI, business intelligence and analytics framework that persuades executives with data storytelling. She fills us in on what the top BI tools are on the market today, and the BI trends she\'s observed that could predict the most popular BI tools of upcoming years. All right, let\'s jump right into our conversation.Mico, welcome to the SuperDataScience Podcast. I\'ve been trying to get you on the show for ages, and now you\'re finally here. Where in the world are you calling in from? I\'m from hot Atlanta, Georgia. I\'ve never been to Atlanta, but I\'ve heard good things. Yeah, I mean, we have a lot going on here. I don\'t know if you saw, we recently got the FIFA and the Olympics coming here, so like, construction is in and traffic is insanity. Not that we need more, but like we, we\'ve got, we got awarded both, so it\'s- Yeah, I do know- It\'s [inaudible] here.Jon Krohn:It\'s the longest commute in America on average. So I\'m like you, like we were talking before the show. I live on one road and I just go back and forth. I, I\'ve, you know, in Atlanta you have to do it for your sanity. Yeah. So I had you recommended to me as a guest by several people, and I think the last person of at least three people that recommended you was Kate Strachnyi, who\'s a wonderful person in the data community. And so on the third recommendation, I was like, all right, we\'re doing this right away. And then it took a while to get you booked. You\'re a very tough, a very hot ticket in town down in hot Atlanta, but we\'ve now finally pulled it off and you\'re here. So you\'re the host of the Analytics on Fire podcast. Yes, sir. It has been, and it may even be still today, the number one podcast in analytics. You are in the top 2% globally across all podcasts, including like all the mainstream podcasts out there, which is wild. And you have somewhat of a cult following. So I understand you can tell us a bit more about this, but when you go to conferences, you have people coming up to you. Yeah. Tell us about it.Mico Yuk:Yeah, so Analytics On Fire is a cult. It\'s no longer a podcast. That\'s what we call it, a cult. Then, because I will go to conferences and because we didn\'t produce like t-shirts and cups, people will print their own things, come up and be like, "Hey, Mico, like, I\'ve been watching you for like, how many years and I have my cup with me cuz I knew you were here." Or their t-shirts or like, you know, I have people that will like walk up and many times and be like, "Oh, when I started my job, your podcast was mandatory to my job". So, you know, because we\'re like your podcast, we\'re like, you know, educational. We bring in leaders, taught leaders like yourself, and they talk about topics that are relevant that can really help people. So the whole premise of it is that we do no bs. So there\'s no fluff on our podcast. We tell it as it is good and bad. So I think people appreciate that, you know, there\'s no marketing. Yeah. You are famously no bs. That\'s one thing that I know about you for sure. Yeah. Perfect. So speaking of educational, you also co-founded an organization called BI Brainz, like Business Intelligence Brains, but the Brains has a "Z" on the end, or Z depending on where you\'re in the world. Yeah, we won\'t talk about The Jerk Off. That wouldn\'t sell us the one with the S cuz he\'s known name. Oh, okay. I see.Mico Yuk:[inaudible] All right. S or not, BI Brainz has been a big hit. You\'ve had 15,000 students use some kind of education from the platform. So you have online students, but also in-person trainings. And so you\'ve had seven-figure revenue for the online course alone, eight-figure revenue if you include the other kinds of training. Consulting, yep. Yeah, consulting. And so I understand that a big part of what BI Brainz teaches is your BIDS framework. This is the B I D S framework, which is the BI slash analytics data storytelling framework. And so the idea here is it\'s a framework for allowing data storytelling, data visualization to persuade decision makers, right? Correct. So I started off as a consultant and enterprise quickly rose up to top 1%. So I worked in New York, I think I told you that on 42nd and second quickly became like top 1% in the world in that, in that field. And then realized that like I went through so much that I wanted to help other people in the enterprises basically go through what I didn\'t go through. And so I actually developed a framework to help people that were like me. Didn\'t know it become such a hit, let\'s be honest. Like I put it out there and I was like, oh, you know, I\'m going to put it out there before, you know, community and all this stuff. And then, you know, who knew there were so many people like me that were really struggling with this. You know, just we were, we had data visualizations, we had reports, but it\'s like, how do you tell the story? Like, how do I get Jon to make a decision because putting a chart in front of Jon is not working. Right? And so crafting that story and created a framework around that, particularly for enterprises where there\'s all these complexities and diplomacy and all that stuff that became a hit. So it was, you know, it was even surprising to us, to be honest. So yeah, tell us about the framework. Break it down for us. Yeah. So without advertising, it\'s very straightforward. So one of the things I noticed Jon, and you\'ll appreciate this cuz you know, you\'re in a decision making role, right? You\'re an executive, is that a lot of times, you know, when you come to need and reporting, people tend to focus on two things, right? They focus on the data and they focus on the medium, the output, whether it\'s the visualization or the report. What I tend to realize, and what I learned was, before you get to all that stuff of visual and data, you actually need to understand what the hell the person wants. And so what I did is I created a framework on three tenants, which is the first one is what you ask. So when you\'re working with someone, what are the exact type of questions you need to ask and not ask?Then it\'s what you write. So the second part is about actually how to storyboard and we\'ve created our own storyboard, and then it\'s what to, what to visualize or draw, right? So then it\'s what, how do you express and communicate that? So we kind of went backwards, but that was again, just based on my experience consulting with Fortune 500 companies around the world. And that has been a hit. We\'ve had multiple companies adopt it from Shell to Ericsson to Nestle, you name it, across the globe. They\'ve brought it in. You know, a lot of time we get data leaders or BI leaders that will bring it in, adopt it, and it it works with any BI tool. So we\'re tool agnostic.Jon Krohn:Oh, gotcha. So some people, I guess the big kind of tools in the BI space are Tableau, Power BI- ThoughtSpot. Correct. And so with us, it doesn\'t matter, right? I mean, the reality is that, and you notice this, Jon, all these tools have a bar chart right? Now, you know, one of them may take two clicks to get a bar chart, another one may take three clicks, but you know, they all have a bar chart. And so we stayed away from the tool and we focus on a framework and whatever output you have, you can actually build in any tool that you want. Because you notice the tool, the tool, this is like the Fortune 500, right? It comes and goes, I mean, in the next three years it\'s all going to change again, right? Today it was Tableau, first it was Business Objects, then Tableau, now it\'s Power BI. Two to three years from now, it\'ll be, you name it, you know. So we, it was very smart for us [inaudible] tool because I think tools come and go, but you got to have the foundational knowledge. Nice. Do you have from your perspective there, do you have any insights for us on what the next tool might be? Yeah. So I have some thoughts. I get asked. That\'s actually the number one question I get asked. Oh, really? Ah, I was like, ah, I\'ve got such a smart question. I bt no one thinks so much. No, but I used to never answer it across the world. Like it\'s literally, I\'d be in these keynotes, I\'d be at these like customer events, like at the vendor event, and they\'re like, Mico, what tool do you prefer? I\'m like, oh, f***. I\'m like, well, it\'s not the tool of the vendor I\'m speaking at. So, right? So I was like, I refuse, I plead the fifth. But I don\'t know, you know, I have some hope for ThoughtSpot, I think they\'re doing some super cool stuff. They\'re kind of ahead in NLP and, and kind of how they approach it. There\'s some contenders up and common. You know, there\'s this Headless BI concept now. Companies like GoodData. My good friend Ryan Dolley works there. So there\'s some stuff, but I don\'t know, I think Power BI is going to be the king of the land or queen for at least another three to four years solid. I just don\'t see anybody that\'s close enough. And I think Tableau\'s going a different direction. So if anybody may be ThoughtSpot, maybe, maybe a GoodData. So I have a few questions coming out of this. So first of all, what, what is Headless BI? What does that mean?Mico Yuk:Okay, so I know you\'ve heard about Reverse ETL, right?Jon Krohn:Yeah, yeah, yeah. But maybe you can break it down for our listeners. Okay. Well, I am no specialist in Reverse ETL, but I could tell you what it is, only because I know the two companies very well that do it, which is Hightouch or Census. I think you\'ve heard of both of those. So essentially Reverse ETL, what ETL used to be was you would have to take the data and get it into one place, right? And then when you get into one place, you would do all this stuff there and then you could use it. Reverse ETL is the opposite. Reverse ETL is, you know what, we\'re not taking the data in one place. We\'re going to put everything, we\'re going to take all the data sources and let you kind of pull from them, pull it together, pull from them as needed and do what you need to do. So essentially what it does is it kind of eliminates this kind of, you know, single source of truth I call, you know, I call, so I can go on a rant on this. I live through the data warehouse era. I hate to age myself, right? And I call it data warehouse era The Lord of the Ring. Why do I call it this, what I call it The Lord of the Rings? You remember the ring in Lord of the Rings? Yeah. It was the precious. That is what a data warehouse is. Yeah. Do you remember how he went down the dark tunnel to try to get it ? He never came. I see, I see. You get it. Right? So I would look through the data warehouse everywhere. Everybody wanted a single story to the thruth. And so it\'s kind of funny today to see that they\'re like, you know what? The hell with that. Stop, stop. You no longer want that. Just give us our data. And so I think out of that, it has bred stuff like Reverse ETL and "Headless BI", right? So a company like GoodData, and it\'s, I hope I get it right. Ryan explained it to me. They\'re not focused on having a encompassing, like a Power BI BI tool where they then hold your data and then you have the visual end, right? They\'re focused on allowing you to take the data and you can actually connect it to any tool that you want, right? So it\'s a Headless BI, it doesn\'t have its own "face" like a Power BI or Tableau. Does that make sense?Jon Krohn:Gotcha. Gotcha, gotcha. So it\'s kind of more like an API than a UI. That\'s exactly correct. And don\'t get me wrong, they do have a visualization element or element to report on, but you don\'t have to use it. So they allow people with all these different data sources to kind of be a bit more free, I guess, heterogeneous in terms of their approach in how they want to visualize, right? I think, I think it\'s a result of the field data leaks. It\'s a different discussion, it\'s a result of the data leaks that everybody wanted like five years ago to fail. Yeah, yeah, yeah. Right. So now we\'re headless, we\'re like Headless BI, so yeah. All right. Pretty animated there. All right. I\'m going on a runt. Going back, you mentioned the Power BI you think is going to be queen for a while. So what is about Power BI that makes it the default choice today? So it\'s not a great tool. Okay. Power BI is not a great tool. I just, and forgive me, Lord, I have so many friends at Microsoft- That must be the first Microsoft tool that people don\'t like. Right? No. Yeah. Okay. No- I don\'t have any, I\'m like, if people haven\'t figured this out from the show for, I don\'t have any Microsoft software anywhere. Okay. I\'m an Apple girl, so I think I have Microsoft on my desktop, granted, but the licensing is always an issue cuz they can never get it right. But anyways, so Power BI is not a great tool. The power of Power BI is the fact that you get everything that comes with it, like Azure, right? So I think what they have done though is that they\'ve allowed Power BI to show up in Office 365 where anybody can download it, but it\'s a visualization tool coming from like a sophisticated tool like Business Office of Tableau, like I did. Power BI is like chop change, baby change. So, you know, I love Microsoft, oh God, let me help you, Microsoft people I love you. I was a Microsoft RD and a [inaudible] I need need to put this out there. I love you. They do amazing research. They actually, they publish more than any of the other big tech companies.Mico Yuk:Well, I call them the Mercedes-Benz of our industry, right? Mercedes-Benz has more patents than anybody else, so I consider them to be, but what is good about Power BI is Microsoft. Microsoft\'s innovation cycle and the way that they\'re innovating. That\'s why I would get Power BI, not because the tool is great, because they are not stopping, you know, you could see how quickly they jumped into OpenAI. I mean, they were on it before the thing got hot, you know? So I think with Power BI what you see is people see that Microsoft is leading as a company and they realize, hey, this tool one isn\'t going to disappear and it\'s going to continue to evolve.Jon Krohn:Gotcha. So it\'s the queen because Microsoft evolves so quickly, they- Innovation. Integrate things so quickly and it also gives you great connectivity with Azure. Yeah. Like take an example, the Python thing, Microsoft bought R, the Power BI team saw Python was running. Yeah. They bought, they bought R. They bought R? I believe so, yeah. Yeah. I think they, I think they, they bought R, they did a stake in it. They identified that Python was, Python was hot. And even though R was a big investment, they just, they just opened the door to Python. You know, they, they are, they\'re very forward thinking. Oh, yeah, I see. Yeah. So Microsoft hired Revolution Analytics- Correct. In 2015. And yeah, Revolution was a commercial provider of R software. Yeah, yeah, yeah.Mico Yuk:Correct. Right. And so think about that. They made this big investment and because they thought that was the way forward, again, they were ahead of their time, saw Python pick up, and I was there when, you know, when that decision was made and they said, you know what, investment or not, we\'re opening the door to Python. And so Power BI fully works at Python. And so, you know, that level of thinking where you\'re willing to be heterogeneous and even cannibalize your own products, that\'s the company that\'s going to thrive.Jon Krohn:I guess with how big they are, they\'re going to have to be cannibalizing.Mico Yuk:Correct. Butchering the process. Cannibalizing, Cannibalizing, they\'re going to have to cannibalize something. Because they\'re, they\'ve got tentacles in everything. In everything, correct. Yeah. So I just think Microsoft is a winning, they\'re winning all around and I don\'t, I don\'t see that stopping anytime soon. I mean, you know, you have Salesforce, you have Google with Looker, you know, at one point I thought Looker was going to be up there, Google acquired them. You have Tableau, Salesforce acquired them. I mean, what, what do we really have left? You know, I don\'t know if ThoughtSpot is going to get acquired, you know, micro strategy has been hanging around. No one will buy it because of the f*** up CEO. So yeah, it\'s, you know, it\'s, is this, should I not say that on this podcast? Sorry. No, I\'m glad that, it\'s awesome to have you be so open about, yeah, I try to, I guess I\'m too diplomatic maybe too much of the time. I should take, I should take a note out of your book. I love it. Oh, you didn\'t see my last post telling people "Sionara [inaudible]"? Oh yeah. I\'ll have to say that went viral. You didn\'t see it? Yeah. I posted something controversial and I lost like 150 followers, like within hours. And they were all like, the top viewers were CEOs, founders, directors, executives, and I said, "Sionara [inaudible]" And it went viral.Jon Krohn:Oh my goodness. That\'s wow. I mean, after the pandemic, like life is, you know, it just, it\'s got to be me. Well it\'s great to have that on this show. Thank you. Mico. Yeah, so f*** up CEO, Michael, I mean he, Michael Saylor, like he\'s all Bitcoin also, you know, it\'s, if you want to do Bitcoin, you can invest in market strategy. Okay. Yeah. I mean, so this is all news to me. But okay, so we\'ve talked about the frameworks or sorry, we\'ve talked about these different BI tools and got your great insights on them. I actually dragged you over here on a tangent. What we were talking about was your state, your BIDS framework- Yes. The BI and analytics data storytelling framework. And so you were telling us about it, you were saying that you break things up into three categories of questions. And so there was what kinds of questions to ask, and then there\'s how do you visually communicate? And I didn\'t catch what the third one was.Mico Yuk:And it\'s what do you write? So we go from asking questions, taking the answers, and put them in the storyboard. So that\'s the written part. And then we take that and we visualize and communicate it. Because what I found, what I found in, you\'re in New York, right, Jon? Yep. Okay. So you\'re known in New York crowd. It\'s a tough crowd. Like it\'s, I started my career in New York and I got my [inaudible] handed to me every day. It was like clockwork. I got, I went to work, got my [inaudible] handed to me, went back home, looked at it, cried, went back to work, got it, handed it to me again. Right? And so and so what I learned, what I quickly learned in New York taught me that lesson was that I was asking the wrong questions. So I would open my mouth and it would be like freaking like again, Lord of the Rings, like it would just like open the door to all these crazy responses that I couldn\'t control. And so what I learned was I break the framework down into specific questions that you ask to get the right answers. So that, that was, that was a big starting point of how you actually get the conversation under control. Nice. So I mentioned how you are, or you were a co-founder. I guess you, I guess you\'re a co-founder, somebody\'s co-founder forever. It\'s present tense. You are a co-founder of the BI experience group that does the education around this framework and that has all these students and seven figure eight- They\'re amazing. But that no longer is your focus. And actually your most recent role was at Count as their chief data evangelist, but you\'re also, you\'ve stepped back from that for the most part. You\'re just doing a little bit of advisory. Did it to a year. Now. Yeah- Correct. Yeah. And so now you\'re looking for your next big thing, at least at the time of recording. Yes. And so, yeah, tell us about that search. I know from a post that you recently made, I\'m going to include a link to this post in the show notes. But you mentioned how when you scroll through your newsfeed and you see who\'s being affected by layoffs, seems to be disproportionately affecting some groups. Correct. Yeah. So this is, again, not to me like, so just so we\'re clear Count is amazing. I was not treated in any way discriminatory. I need to put that out there because some people wrote me, I said, what did Count do you? I\'m like, Count did nothing. My, our CEO is amazing. I was treated amazingly, but this is just my observation and I\'ve been watching it now for months. So if you go to tech layoffs, anytime you see tech like Shopify layoffs or another product, any company, just any of them just roll down. And what you mostly see is either women, older people, or just minorities as a whole. Okay. And so I I, I\'ve been tracking this and I kind of started, I looked at it and I said, well, you know what, this is not conclusive, right? I was a data scientist, so I\'m like, you can\'t just look at a LinkedIn.Maybe women are more expressive, you know, and maybe they just go online and post. So I decided to start to speak to people and unfortunately behind the scenes, I had gentlemen, white males confirming to me that, Hey, something does look a little stinky, you know, in multiple companies. And I said, well, you know, like, what, what\'s going on? And they said, well, there\'s so much happening. It like, it\'s happening so fast you only could see it after the fact. But I did actually go back in and confirm through a few companies with no names Paul, that like, you know, some people are watching this going, Hey, this, this is a little stinky. Like why do we get rid of the women on maternity and why we get rid of all, you know what I mean? So there is some discrimination going, you know, I had multiple women tell me, yeah, I am, you know, 35 or 38 or 45 and all that\'s left is a 20 year old women.I had, I\'ve heard that multiple times, you know, or I had gentlemen tell me that. They\'re like, yeah, I have all this experience. You know, and all that\'s left is one set of age belief as a good example. So I, I, again, not conclusive until you have the data. I was a data scientist, so I\'m sensitive to that. I understand that. But it\'s very visual and it tells a lot. And so, you know, there\'s definitely something happening and I think my biggest fear is that we\'re, you know, three, four years from now going to look exactly how we looked before, like we\'re reversing, you know? And so it\'s just a big fear. But I went viral. Yeah. I just, I just put it out there cuz I was trying to call out to say, Hey, if you\'re reading this, and then I got a bunch of private dms of gentlemen telling me, you know, hey Mico, what you do matters. And you know, thank you for pointing this out. Clearly you don\'t want to be public, but you know, they, they\'re like, what you do matters. You should just know that what you do matters. Don\'t lose your voice. What you do matters. So I know that if it saved one minority in any category, whether it\'s gender, ethnicity, race, or age, it was worth the word, the loss of followers. You know? Yeah. I am aware of at least one big tech company that in layoffs was I think there\'s a lot of constraints against letting people go when they\'re in maternity leave. Right. But so something that they were doing was people who had been approved for maternity to leave, say to start next week. That\'s correct. And that\'s wild. And it, and it just sabotages like somebody took the list and just sabotage it. But no, I spoke to women that were at home with their babies on leave and got the ax. I mean, just [inaudible] got the ax. Like they were scheduled to come back to work next week or da da da and they just, they just told them, don\'t bother to come back. You know? So, you know, don\'t get me wrong. Like I, I\'m not saying again without conclusive data, we\'re data people. We got to be careful that it\'s, maybe it\'s only visible the people who type of people who post, but it, it, you know, talking to people, I got a good perspective that yeah, it, there\'s, there\'s some smelly stuff going, you know, that, that supposedly it\'s random but it just keeps all looking the same. Yeah. Well, so there\'s a good project out there maybe for a listener where something that I\'m sure I\'ve said on air before is that, you know, if you\'re looking for maybe your first data analytics job or your first data science job, having your own portfolio of research on some, independently on some project that you independently conceived of. So good. So this could potentially be one where- Agreed. Look into this problem and see whether this is in fact you know, something that you can, that bears out in the data and yeah. If that is the case, then please tag Mico and me in a post about that. Yeah. So anyway, so that was just something that I saw again, not hopefully not necessarily affecting me though I did have an interesting experience. Yeah, I\'m out there looking for my next opportunity. I am interested in helping to build communities. I\'m very, I\'m very gunned up by how big DBT got on Slack, you know, that was very exciting for me. So I\'m pretty excited, you know, not the developer advocate cuz I\'m not as technical anymore, but anything at like drives community, events, engagement, that\'s kind of where my sweet spot is. I\'m a community like you a media person, community person. Right?Jon Krohn:Well no doubt- [crosstalk]. Yeah. You\'re going to find an amazing cult to lead next alongside the cult that you continue to lead the AOF fam analytics inspired fam cult. After I just defamed like five companies on the podcast. But yeah, that\'s fine. I mean- Yeah. It\'s not going to be every company. So Mico, this has been a really fun episode of course. And we\'ve also learned some great perspective on the BI and analytics landscape. So before I let my guests go, I always ask them for a book recommendation. Yeah. So this is definitely not your typical data book. There\'s actually two books that I like. The first one is The Richest Man in Babylon. It\'s by George Clason. It is epic. I heard it first by, it was recommended by Noah Kagan from AppSumo, I\'m assuming you\'re familiar with AppSumo. And he recommended it. And so I took it and I did the audio version, which, you know, audiobooks always weird, depend on the narrator. It\'s such a good book. It\'s kind of like Slumdog Millionaire style, but the lessons that are learned are amazing. And if people like that book, one of my other favorite books I\'m reading right now is The Psychology of Money. Those are both like epic books. Psychology of Mind? Of Money. Of Money. It\'s epic. Yeah. So those are both, you know, and, and so much more not money, obviously I know they both talk about wealth, but the way in the style the books are written, it\'s so much more deeper than, you know, just financials, right? It\'s just overall mindset because, you know, financial wealth is a journey, right? It\'s not a, it\'s not a marathon, it\'s no silver bullet. So those are two books I\'d recommend. Nice. Very cool. Great practical and economical tips for our audience there. Thank you Mico. All right, so I imagine there are lots of listeners out there who want to be able to follow you after this episode and get more real talk. Except the vendors. What\'s good, what\'s bad? No punches held back. So obviously your Analytics on Fire podcast is a way for them to go. How else can they follow you? Yeah, so most people follow me on LinkedIn. It\'s pretty easy. Micko, Yuk, M I C O Y U K. And then recently I opened up my Instagram account. I\'ll be doing more on there. So feel free to follow me. And as you know, I\'m getting ready to start my YouTube channel, so any of those channels will be great. As of today. I do a lot on Instagram. On LinkedIn, but you took [inaudible] Nice. We\'ll be sure to include links to all of those in the show notes. Mico, thank you for finally being on the show. It has been an absolute hoot to have you on and hopefully it won\'t be too long before we can get you on again. Yay. All right, well hopefully, hopefully you don\'t have any sponsors that get pissed off, but yeah, I\'ll be happy to come back. Thank you so much, Jon. Thanks Mico. What a switched-on person Mico is about the analytics industry. In today\'s episode, she covered her BIDS, BI data storytelling framework for persuading decision makers. She talked about how Microsoft Power BI is the queen of analytics today while ThoughtSpot could be the heiress. And she filled us in on how Headless BI tools like GoodData allow for Reverse ETL that\'s extract, transform, and load operations. All right, that\'s it for today\'s episode. Until next time, keep on rocking it out there folks and I\'m looking forward to enjoying another round of the SuperDataScience Podcast with you very soon.'
doc = 'Welcome to the SuperDataScience show. For this special 500th episode, we\'re going to enjoy a mind-expanding Yoga Nidra session led by the wonderful Yogi, Jes Allen. Jes Allen, can\'t believe you\'re on the SuperDataScience podcast. It\'s wonderful to have you here. I was thinking for a while, I was thinking for months when I noticed that episode 500 was coming up, and what was months ago a distant future. And now it\'s not that far away, episode 500 will be released very soon and we\'re recording it right now. And so I was thinking to myself for months, I was like, "What can I do that would be extra special for the audience for episode 500?" And time and again, the idea that kept coming back to me was you offering a yoga Nidra practice. So now here you are. Yeah. Thank you. I\'m so excited to be here. I\'m so happy to be here. So I\'ve known you for a long time. I started doing yoga practice, the poses of yoga in 2012, I was living in Singapore and once a week on Saturdays at this mixed martial arts gym, I was a part of, they had this yoga class. And I was sore a lot from all the mixed martial arts, And so I was like, "I really need to stretch." So stretching, that\'s what yoga is. And we\'ll get into this, but yoga is a lot more than stretching, but that\'s what I thought at the time. And so I went to the Saturday class and I remember the opening up of my body, literally, I mean, the physical postures are helpful for opening you up. I remember feeling amazing in a way that I couldn\'t remember having felt later that Saturday, later that day. So I went back, started going back every Saturday. Left Singapore, moved to New York, and not long after that, I went to a yoga studio in Manhattan in NoHo and you were one of the teachers there. And I had a class with you that was incredible, completely beyond the quality of any yoga class I\'d ever had before. And at the end of the class, you said, "All right, well that was my" ... I can\'t remember what the classes were called.Jes Allen:They were really simple names, like strong, energize, relax. Right. It wouldn\'t surprise you even with what I\'ve just said, and certainly just the way you know me, but I was there for the strong classes, the strong yoga. So I would come for your strong class and then at the end of it, you\'re like, "At 10 minutes or something, I have another class starting, a relaxed class in the same studio." And I was like, "Oh, alright." And I stayed for that. So it was like three hours or something of yoga with you just to kick things off. And now I\'ve been following you around the world. So that was eight years ago. And I\'ve gone on retreats with you to Mexico, to Greece and have been to classes at studios of yours, studios that you\'ve taught at all over New York. So anyway, huge fan. And I know episode 500 is going to be amazing. So in episode 450, we had Steve Fazzari whom you\'ve met once, he came to a yoga class of yours once. And Steve talked to us about the physiological benefits of yoga nidra, like improves sleep, for example, is one of the things that he talked about. And at the end of the episode, he did a quick five minute run through of kind of like the structure of a yoga nidra practice. And so for today\'s episode, we\'re going to do a full practice. But before we get to that, I\'d like to talk a bit about what yoga means and what it means to be a yoga practitioner. So as I\'ve now learned in the last decade, yoga is not just stretching, and yoga is not even just the physical postures. So what in yoga are called the asana, the physical postures, that\'s part of it. But I mean, just tell us more generally what yoga is all about. Well, I have to comment on your language of stretching and opening, because even those two words, you are referring to them in your physical form, but it is still, they\'re expansion words, they\'re words that also mean to expand. And yoga, it means union or to yoke, but it\'s to become unified consciously with the one consciousness, it\'s to remember that we don\'t have to come together because we\'re not apart. We don\'t have to unify, we are unified. And so we expand our awareness of consciousness through practice to remember that we are consciousness ourselves, that the essence of us and all things is pure consciousness. So as you stretch your body, you stretch your consciousness. As you open your body, you open your consciousness. You expand, you expand your awareness of yourself, of how you hold patterns, of what it feels like to be in a human form, of how you relate to emotions and programming. And as you become aware of yourself, you start to become aware of your interactions with everything. So even starting with the asana as a practice is incredibly expansive. And I think we all experience that in the beginning, we\'re like, "Wow. And I feel more connected to myself." And life starts opening up because I\'m opening to myself, which is how we experience everything. And then at a certain point, our curiosity, if we feel called to continue to expand our consciousness, takes us into different states of practice. And I think ultimately, as starting also as a very strong yoga asana practitioner, and then in my journeys over the past decade, coming down to settling into a practice such as yoga nidra, which is laying on the ground doing nothing, there has been quite a journey. But this is where the expansion of consciousness and understanding that has taken me ... And I think it\'s just important to note that we\'re not expanding consciousness, just our awareness of consciousness. Consciousness is always expanding on its own. So these practices bring us into relationship with the natural expansion of life, as opposed to feeling like it\'s something that we are separate from. A beautiful definition. So in this decade of practicing yoga, you started off teaching these kinds of athletic classes. There were meditative elements. It was through my early yoga practices, and particularly through your classes that I started to be present in the moment in the room, in my own mind for a few instance. And I remember thinking, being on a yoga mat, doing a yoga flow and being like, "Oh wow, I\'m here. I\'ve got hands. My hands are on the mat, there they are." And not being caught up in the next thing that I need to do or things that I\'d done that day that I could have done better. And so anyway, so you started with the physical, teaching the physical asana practice about a decade ago, and that has now transformed, as you\'re saying, to include things like yoga nidra. And so yoga, it can be a big umbrella term to capture the physical postures like asana, nidra, like you said laying down, and surely not doing nothing, but not moving. You can think of it as like a philosophy of life. There\'s teachings, there\'s literature that you can read. I have experienced that. I\'ve been to your apartment in Brooklyn and studied the Upanishads which are Hindu scripture and studying those week after week with a group of people. And there are books like that, the Bhagavad Gita is a popular one in yoga. And so there\'s these books that you can use to learn about yoga as a way of life, as opposed to just postures. But what I\'m specifically getting to is today, what do you offer as a yoga practitioner? So we\'re going to talk about yoga nidra a lot. So that\'s obviously something that you offer and we\'re going to experience on this podcast. You also have weekly online asana physical practice. I know you still do retreats. There\'s one coming up in Greece in September. And it sounds like I\'ve also heard you recently been getting into coaching. So maybe tell us about all of those things. Sure, yeah. So I guess I am an intuitive being and we all are intuitive beings. And I\'ve always related to myself as an artist first growing up, this was really what it was for me. So my path with yoga and practices have been really circular and I\'m a curious being. So I actually was a gymnast growing up and I took my first yoga class when I was 17. And it was a little slow for me at the time, but I remembered understanding it because of the postures and creating sequences and having body awareness that I learned. And I understood it to be a self-discipline even at that age. And there was something magical that clicked for me in the opening of awareness. And like many of us at that age, I was in a pretty nodded relationship with myself. And I did my first teacher training when I was 21. And the most amazing thing for me in that training was learning about the chakra system and the energetic systems of our bodies, because this is how I really was. I was looking for the magic, right? I didn\'t want the answer in a scientific way. I wanted to understand life the way I was understanding and receiving it, which was through feeling and awe and magic and alignment synchronicity. So I had a rough start in my healing, my early 20 healing years. And when I finally really came back to yoga, it started with the asana, and it was so important for me at the time because it gave me confidence to change my life. It showed me that I could learn things and build strength and figure out tricky asanas. And it empowered me to go, "I can change my life. I can figure these things out." And that was really important to me then. And since then, basically I started practicing and teaching at the same time, in that way. I had been practicing for a while, when I came back to it, I was like, "This is what I have to do. I have to teach." And so my practices in my teachings have gone like this. So as I have ... One in front of the other, like little steps, like a little track. And so I would have an opening in myself and feel called towards something. And then once I would feel like it was embodied in my being, I would want to give it back. And so one of my favorite things to do, and what I started doing early in my teaching was cultivating my own classes at different venues in New York, in the early days in Brooklyn. And then I started leading retreats. So retreats have been just the most magical experiences for me to create over the past five years. I\'ve led, I think, 17 retreats in that time. And it\'s just the most amazing thing to bring. As you have felt a couple of times, a group of human beings together in a particular place, for a particular amount of time and just really dive in and like magic is super available. So that is a forever passion of mine and I\'ll always be offering those. It\'s tricky in the world right now, but we are going to Greece in a month. And I\'m working on Bali or Morocco over the winter. So that\'s happening. Just talk quickly about the retreat experience. I don\'t know, I\'ve only done yoga retreats with you, so I might not have a really good understanding of what they\'re like. But I might have thought prior to going on them with you that a yoga retreat was this kind of intensive physical practice. And of course, I expect the same kinds of like you\'re saying, as I open myself physically, I\'ll open mine and I\'ll have this amazing positive experience, getting just a good break away from the world and do lots of yoga with cool people and talk to them. But, the way that you structure your retreats, and it sounds to me like the way you increasingly structured them is so that yes, there is a physical practice. So probably twice a day. So there\'s like a morning practice and evening practice on most days, but there\'s tons of other deliberate practice of journaling, of answering tough questions together in a structured way, of getting to know what people are like in a structured way and what your own mind is like and what you are like and what you want and what you could be doing and what you could do in the future. And, yeah. I mean, I guess I\'m trying to make a case for yoga retreats in general, and yoga retreats in particular with you. But if you want to get away from the regular patterns that you\'re in, in the other 50 weeks of the year and spend a week or two of your year, just taking a step back and thinking about, "What am I doing?" Well, that\'s what retreat means, right? It means to step back. It means to withdraw yourself from a moment. And I know that you meditate every day. And one aspect of meditation is just in the beginning, to just become a witness to your mind and to your life. And we don\'t give ourselves enough time to really see how we\'re living and see how we\'re seeing. And so when we intentionally remove ourselves, we can go on vacation and that can feed us in a million ways. And when you retreat, you drawback consciously and engage in inner practices, things I\'m doing now are like asana in the morning, meditation, yoga nidra, again, sound, cacao ceremony, silence. All different kinds of workshops to awaken you. And this really has been my path with practice is finding all these different things to serve as mirrors to who we are. I have told myself that I\'m on a path of many mirrors because this is so exciting to me to go in. I started with asana and at a certain point ... And then the energetic systems. And then I moved into chigong, and I moved into meditation, and I moved into cacao and journaling and different aspects of meditation and yoga nidra and more. But each of these little things showed me a different perspective of myself, a different piece, a different way to see myself, a different way to experience myself. And this helps us see the world in that many ways. If I can see myself in this many ways, I can only access in another being as much as I\'ve access of myself. I can only access in life as much as I have accessed in me. So the more I understand how diverse and expansive I am, the more diverse and expansive my life can be. And something I want to just highlight is that when we talk about yoga and we talk about expanding consciousness or expanding our awareness of consciousness, it\'s like, "What\'s the point? What\'s the point in all of this?" There\'s like a lot of big, fancy words and that\'s cool to be spiritual and everything, and like, "Yeah. I\'m one, and [inaudible] conceptually." But what is the point? What\'s the point in doing practice? What\'s the point in showing up every single day? And then we get into this big word, which is healing. Healing is the point. We are wanting ... I think all of us are wanting to live a life of less suffering. And we are part of a collective evolution that is at the pace that it is at. And the first thing we do is heal ourselves. As we heal ourselves, we create a ripple in consciousness that continues to stimulate awakenings all over the place. And that\'s how the one organism of humanity wakes up. But even just for us on an individual level, if I can observe my life without so much attachment to outcomes, expectations, stories of the past, things that I want, things that I\'m not okay with, if I\'m able to loosen my grip, can I live with more grace? Can I feel more joy? Can I feel more gratitude? Can I allow things that are uncomfortable for me to come through my field and move out without completely destroying my day or longer than that? And so these practices are really like, can you associate yourself? Can you learn to relate to the part of you that is unchanging? The conscious witness that we all are, can you relate to that? Enough that the fluctuations of life and the mind no longer with you around like a ship out in a storm. And when we can do that, we just naturally become more loving and more compassionate and more understanding of the fluctuations of everything around us. So if I can show up and say, "I\'m working myself towards understanding the one consciousness and surrendering over to an evolutionary current that I am not in control of, but I am a witness to. Well, now, I\'m so free." I\'m so free to step into life and be forgiving of everything that I can\'t control and return to what are the universal truths? My teacher Yogarupa Rod Stryker, who I received my yoga nidra training with that I resonate with the most, he says, "If it\'s not true for everyone, it\'s not true." And this is something I come back to, if it\'s not true for everyone, it\'s not true. And that\'s not to say your experience isn\'t valid, your feelings aren\'t valid, all of them are true and valid, but what is the ultimate truth? And can I remember that peace in myself so that I\'m not only just Allen who lived in Brooklyn for 12 years and just moved upstate and used to teach there, and now does this, right? I\'m not only those things. Which is challenging and confusing to be my emotions, my actions, my relationships, that\'s hard. So I\'m not only that. All right. So obviously your retreats sound amazing. Yoga retreats in general sound amazing and can be hugely fruitful, but not everyone, especially in a pandemic can get up and fly to Greece. So how are ways that people can engage with you to get mentorship in some other way? Yeah. Well, one thing that the pandemic has definitely opened up, it has been the online platform. And I started a Patreon account really as like a response to all of the yoga studios getting shut down about a year and a half ago. And it\'s been such an amazing opportunity to share content. And so I\'ve got that platform that has tons of asana practices, yoga nidras, meditations, some workshops, and I get on and do lectures every once in a while. And what that has really opened for me is this understanding of empowering the practitioner at home so that we aren\'t actually relying on going to a yoga studio every single day, or even going on retreats which is something that you might only be able to do once a year, if that anyway. And what has become very apparent to me is that what we need is more practitioners as opposed to teachers. There\'s a lot of teacher training programs out there, and that\'s great. And also you spend a good amount of time learning how to teach in a teacher training, which might not necessarily be what you want to do. And I believe that we need to all be in practice. This is how we are waking up to ourselves and to our life. And so I have recently started mentorship for practitioners. So I\'ve been doing mentorship for teachers for many years, newer teachers, teachers who want to go deeper, and that is really amazing. And I love to get into that work from that teacher mindset. And I\'ve started in the past few months doing mentorship for practitioners. So basically that looks like us meeting and dissecting, like, "Let\'s look at what\'s happening in life right now. Let\'s look at what you\'re feeling on creating, expanding, opening within yourself." And then the past really like decade or more than that, of me being super living and practice, living and practice. And navigating my life through practice, I\'m able to help you build a practice for yourself. And navigate where you want to go and give you the tools so that you feel empowered to be able to take these tools into your own life, even when I\'m not around. And that\'s really the goal is to empower your own intuition and feel like you have enough tools and enough knowledge that you can care for yourself and that you have antidotes and medicine in your life to come to every day. And this has been an amazingly rewarding practice for myself to offer in this kind of way and see people start to blossom and just get so excited about practice and empower and empower them to take control in their own life and on a regular basis. I don\'t go a day without practicing. I can\'t imagine what that would be like, however, it doesn\'t look the same every day. And that\'s important to know too. That sounds really cool. And so a yoga practitioner can be anyone, right? So a yoga teacher, we kind of think of that as like, "Okay, maybe you need" ... I mean, you don\'t actually, but you kind of think of it as needing a formal qualification and then being qualified, stand in front of a room and talk to people. Being a yoga practitioner means just practicing the teachings of yoga and experiencing more connection with everything around you. Absolutely. And this is the key to us. We\'re not all going to be yoga teachers. We\'re not all going to be meditation teachers and spiritual teachers, but we are all beings of yoga or beings of spirituality. And we can be that. Beautifully said Jes, and I\'m looking forward to reviewing the recording and experiencing everything you just said there again, because that was awesome. I guess we should get into the yoga nidra practice. I mean, I think we\'ve teed it up really well now. So we had a bit of an introduction to yoga nidra in episode 450 with Steve, but you teach yoga nidra from a different lineage. So I think we might as well just hear it fresh from you. So what is yoga nidra? What are we about to experience? Yeah. So actually, the lineage that I am teaching in is the Shri Vidya tradition, my teacher is Yogarupa Rod Stryker. These practices and the interpretation of yoga nidra, the way that he has received them from his teacher and were passed to me are from the Mandukya Upanishad. So straight from the upanishadic literature, which speaks to the four states of consciousness and the actual technique of yoga nidra, and yoga nidra of being both a practice and a state. So when you say yoga, yoga is not just a practice, there are yoga practices, but yoga is a state of being, it\'s the state of being connected consciously. Nidra means sleep, which Steve probably spoke too. And so yoga nidra means sleep with yoga or sleep of consciousness. So it\'s to come into a state where you\'ve rested your entire body, mind, all of that is asleep, but consciousness is still there. Consciousness still exists. So the way I have also been passed information in this lineage is through having a handful of different intentions for the practice. One of those being healing and general healing, which is what Steve really spoke to in the physiological healing of the body, the nervous system, getting immense amount of rest, which is so, so, so, so, so important. Another intention that we\'ll be diving into today in the practice is a cognitive practice, which allows us to associate ourselves with the witness consciousness and then observe the way our mind is producing thought and what is retaining through our days, through our life. And if those things are beneficial to us or not. And then if we want to continue to explore how to work with our awareness to perhaps bring more grace into your life. There is also a transformative practice, which we can take things that are triggering or activating, or that we\'re having challenges with in our life and bring them into this soft state of yoga nidra and activate them in our being. And then ask for resolution or healing imagery not necessarily mental-based conversational solutions, but felt energetic healing for these things to bring ourselves into a state of peace and to be able to work with them with more grace. There\'s also the practice of sankalpa, which Steve may have spoke to also. This is a practice of creating a resolve for yourself that is like an intention that is almost something bigger than you think that you can do on your own. So you call for divine intervention. Yeah. So it should be a really big intention, something around eight to 12, eight to 18 months even. A goal this big, right? Nothing that you\'re like, "I can do that next month." At least a year your calling in this prayer. And then the primary intention and at the basis of all of this is spiritual awakening or becoming unified with consciousness and resting in that one consciousness. So there are several different kinds of techniques to work within these spaces. And they\'re almost a progression. Not that one is better than the other, but that we have to first heal our bodies, understand our minds, transform our perspectives, create goals from a pure place, create fulfilling lives for ourselves from that awareness, and then be able to rest back enough to awaken into the magic of truth. So the way I use these techniques is they\'re all over the place. They\'re not this linear progression that our mind wants to grip onto because we have that past, future ladder mentality, but everything is circular. So they work together. And part of practice is knowing when one can come in and [inaudible] when they shift. It\'s very deep. It\'s extensive. One thing I do want to share that was one of the greatest insights I have received through this work is that when you set up the conditions for healing, healing happens on its own. And we\'re really attached to us being the doer and us changing ourselves, whether that\'s through physical change, we feel like we have to ... I\'m using my hands here because we feel like we have to grip and change and mutate, and that is only our will that creates change in our life. But that\'s not true. We do have to be an active participant, but our soul calls us into alignment and it\'s just our job to walk towards our own soul, our own vision. And healing is really the natural state. So we talk about this as like a remembering of who we are. We\'re not learning something new, we\'re just uncovering the truth. And so when you lay yourself down and you come into a space of unity, you remember. And that healing starts to happen on its own because as your body, not even your mind, but as your body and your cells, your nervous system start to remember that you are safe, you are loved, you are held by this earth that you came from her, that we are connected to one source. All of these other fears and anxieties, they start to dissipate. The more that we remember our truth, the more we see the illusions of these fears. And so we can work into the healing through coming into love, coming into love, coming into love instead of how do I get through this pain? How do I get through this pain? How do I get to this pain? So the yoga nidra practice that we are going to be diving into in this episode is the cognitive practice that I spoke to in which we will use entering the state of yoga nidra as a foundation for us to bring awareness to our mind, our days, how we are seeing, what we are remembering. And get a better perspective, even an eagle-view of our life. So for this practice, you will want to be laying down and be comfortable. Being comfortable is one of the most important things in this practice. But we\'ll give you a moment just to set up, you could lay down on the floor with a blanket underneath you and a pillow underneath your head. You could lay on a couch, you could lay in your bed. It doesn\'t matter as long as you\'re comfortable. It is recommended, although not necessary, if you\'re in a very light-filled room to cover your eyes with a light piece of fabric. And if you\'re cold at all, to place a blanket over knee, over your chest or over your legs. If you are placing a blanket over you, it\'s recommended that the fingertips are exposed. So you can just let your hands slip out the sides, or even create little tents underneath the blanket with your hands so that the fingertips are sensitive and aware of space around them without weight. So just making your final adjustments here, and perhaps even fidgeting a little bit in the body. If there\'s excess energy, you could squeeze your hands and squeeze your feet just for a moment and then exhale and let them go so you can feel that release. And then you\'ll just want to commit in this moment to being still throughout the duration of the practice. If you feel the urge to move or adjust now, just do it so that you can say, "Okay, now I\'m comfortable enough to be here for the next 15 minutes." Just notice your breath. Now allow your body to open to the earth. The back body to become sensitive to the draw of gravity. Perhaps even feeling how deeply held you are. And then that feeling of being held, give your body any amount more over the rest. Become aware of yourself in the space. There is a floor or a piece of furniture underneath you, walls around you, sealing above you, and you are occupying this space, and this space is holding you. Just becoming aware of any sound, allowing your consciousness or your awareness to just travel to one sound and the next not lingering too long on any sound. Rather just tuning yourself to the textures of this space. Bringing your awareness to your breath and specifically down to your belly, and just gently beginning to shape diaphragmatic breathing. So on purpose, breathe into the belly and breathe out of the belly. So the chest here is more or less still, and you\'re directing the breath down into the diaphragm. We\'ll breathe like this for about a minute, consciously shaping the breath. I think two more breaths. After that second exhale, just allow the breath to return to a natural state, just simply observe the breath flowing. You\'ll make an intention for yourself here to stay aware throughout the practice. So there\'s a good chance that the body will sleep and a small chance that the mind could even sleep. But our intention is for awareness to remain, so just say to yourself three times mentally, "I will stay aware throughout the practice." Now please bring your awareness to the third eye center or the space right in between your eyebrows. And notice as you do this, a gentle energy emerges. It might feel like a soft light, but there is a presence here. And just notice that presence that is already existing, but it awakens through your awareness. And then bring your awareness to your throat center and notice that same light or energy emerging with your awareness in the throat. Right shoulder joint, a light or awareness comes from inside the body. Right elbow, same light or presence. Right wrist, light inside the body, energy from within. Right thumb, pointer finger, middle finger, ring finger, pinky finger, all emerging light. Again in the right wrist, presence of consciousness. Right elbow, right shoulder joint, same light or presence emerging from within. Again in the throat center, feel that light. Left shoulder joint, again, a presence emerging from within the body, creating softness. Left elbow, inner presence. Left wrist, light emerging. The left thumb, pointer finger, middle finger, ring finger, pinky finger, full energy and light. Left wrist, presence from within. Left elbow, left shoulder, inner light. Throat center, awareness. Now in the heart center, feel an energy, a light presence emerge with your awareness of the space of the heart. Same awareness over to the right side of the chest. Back to the heart center. Awareness now in the left side of the chest. And heart center, feeling that light. Awareness, again, in the throat center. And landing the consciousness in the third eye center, the space right in between the eyebrows, and just allowing the consciousness to remain. Notice here that the consciousness bears witness to these energy points, and observe as you allow all of these points of energy or light to illuminate at the same time from within. Watching the brightening of these lights. Allowing that energy to keep the body soft, bringing your awareness back to the dark space behind the brows, behind the forehead, the dark screen of your mind. And beginning 30 minutes ago, recall on that screen what you are doing. And then begin to trace back your entire day until this moment in 30 minute intervals. Just allowing yourself to pass over interactions, actions, thought, feeling, right until you woke up this morning. A few minutes to do this. (silence) As you approach the morning, can you see yourself in bed right before you woke up, and recall any dreams from the night, encouraging the mind to continue to move in intervals through the evening of sleep. And then perhaps, becoming aware of the moment right before you fell asleep. Allow yourself now to release the hold of memory and draw back so that you may witness anything that was less than beneficial, or didn\'t sit in the most pleasant resonance from even just this day. And just from the state of witness, gently assess, invite awareness to these areas of your life. With compassion and mere awareness, but awareness doesn\'t judge, simply witnesses. Can you allow everything that you\'ve drawn out of your mind to exist and bring yourself back to the witness consciousness or the place in you that sees. And arrest in this consciousness as self. Without moving the body at all, simply become aware of the breath in this state. And again, keeping the body very soft, bring your awareness back into the room, feeling yourself again, in this space, on the surface, contained. I\'m taking just another moment to hold close to you anything that you\'d like to carry with you outside of this practice, you could even plant that like a seed in your heart. And when it feels right for you, you could very gently move your fingers. Take a deeper breath in. I like to place my hands on my belly or my heart as the first movement to keep things gentle without jolting too much out of this sensitive space. I\'m just feeling my palms and the breath meet each other. Whenever you are ready, knowing that you could just continue to lay right where you are, you could also shift to a side, come up to sit and eventually invite yourself back into your room, the waking part of your life. So welcome back from that exploration. Thank you. Thank you. I just want to note that, again, this is a practice that helps us to just bring more awareness into our life and it\'s something that you could even do as a cognitive practice before you go to bed at night, like five minutes to retrace your day. And it improves memory in general, and it\'s also just really good for maintaining awareness in our life and being able to assess and shift anything that isn\'t really serving us. And when we do it in that state of yoga nidra, we\'re also able to relate to that unchanging piece of us that is it is not our mind, but can work with it. Yeah, I had not done that before. I had not gone through a nidra practice like this before, and I\'ve done nidras with you, but I think it was before you started studying this particular lineage. And I really was blown away by how many things I\'ve already done today. We\'re filming in mid-afternoon and wow, so many things already happened. And you talked a lot about healing and something that occurred to me just now is how happy I am with so many of the things that I did today that I feel like I was kind to people around me and I was taking on kind of the right priorities for my day. But there were also a couple of things that came to mind that I was like, "Huh." There\'s a little thing that like it definitely had been better, that interaction could have been kinder there. And yeah, very cool Jes. It does sound like something that would be super helpful at the end of the day, in particular laying in bed. And maybe even then bringing that piece into my sleep. Yeah, if you practice enough, you\'ll start recalling your subconscious dreams, which that\'s a whole other topic but it has a lot of information, just information about how the mind is processing our desires in our life. Nice. Well listener, I hope you enjoyed this extra special episode 500 of the SuperDataScience show. Jes, how should people be aware of everything that you do, everything that you have to offer? How can people follow you in the world? Well, I have an Instagram, this is the way of the world today. It\'s just my name, it\'s Jes Alan. I do spell my name with one S, so it\'s J-E-S A-L-L-E-N. The handle has little dots in between each of the letters, but you should find me anyway, just my name. And my website is also jesallen.com, and you can find all my retreats, my online content, mentorship programs. I\'ll be coming out with some group programs also soon that will be accessible in the virtual world. So both, all my content and interactions are on both of those platforms and they seem to be best and easiest. There\'s also some free content on SoundCloud as far as yoga nidras, and some meditations go, so you could also start by tuning in there. Yeah. Perfect. Thank you, Jes. And we will include links to all three of those Instagram, website and SoundCloud, which I enjoyed in the run-up to filming this episode as if I needed extra content to be sure that this was the perfect thing for this episode. I did thoroughly enjoy a couple of nidras that you had up on there, on SoundCloud. All right, thank you so much Jes. Have a wonderful rest of your day and we\'ll catch you again soon. Yeah. Thank you so much for having me, it\'s beautiful to be here.'



kw_model = KeyBERT()
keywords = kw_model.extract_keywords(doc,keyphrase_ngram_range=(1, 1), stop_words=stopwords)

/Applications/anaconda/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn(


In [37]:
keywords

[('yoga', 0.5092),
 ('yogi', 0.3883),
 ('yogarupa', 0.3784),
 ('nidra', 0.3233),
 ('nidras', 0.3059)]

In [25]:
stopwords = list(STOP_WORDS)

In [ ]:
def preprocess_text(text):
    doc = nlp(text)
    tokens = [token.lemma_.lower().strip() for token in doc if not token.is_stop
             and not token.is_digit
             and not token.is_punct
             and not token.is_space]
    return tokens

In [ ]:
# https://maartengr.github.io/BERTopic/index.html